# Step 2a Global analysis of 670nm excitation TA of WL-PSI of SCy6803

### Package imports

In [ ]:
from glotaran.io import load_parameters, save_result
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

# Plotting related imports
import matplotlib.pyplot as plt
from pyglotaran_extras import plot_overview, plot_data_overview
from pyglotaran_extras import plot_doas
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths
from pyglotaran_extras.inspect import show_a_matrixes
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras.plotting.style import ColorCode
from cycler import cycler


### Data inspection

In [ ]:
# specify data paths
DATA_PATH_670_1 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_reva.ascii"
DATA_PATH_670_2 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revb.ascii"
DATA_PATH_670_3 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii"
DATA_PATH_670_4 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii"


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH_670_1,
    nr_of_data_svd_vectors=5,
    linlog=False,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
    title="670 nm excitation Time Range 1",
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH_670_2,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=10,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
    title="670 nm excitation Time Range 2",
)


## Global Analysis

### Model specification

This time the model and parameters are defined in each step, as the model and/or parameters may be tweaked in each step.

### Define the analysis scheme and optimize

A scheme is a collection of a model, parameters and data, along with options for optimization.

In [ ]:
global_scheme = Scheme(
    model="models/global_step1_model_PSI_TA_SCy6803WL670_without_DOAS.yml",  # type: ignore
    parameters=load_parameters(
        "models/global_step1_parameters_PSI_TA_SCy6803WL670_without_DOAS.csv"
    ),
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH_670_1,
        "670TR2": DATA_PATH_670_2,
    },  # type: ignore
)
global_scheme.validate()


In [ ]:
global_result_670 = optimize(global_scheme, raise_exception=True)


<sub>For reference, after 14 iterations the final cost is 3.3293e+02</sub>

## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
from custom_plotting import plot_residual_and_svd

fig, axes = plot_residual_and_svd([global_result_670.data["670TR1"]])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


## Plot result for interpretation


In [ ]:
from custom_plotting import plot_concentration_and_spectra

myFRLcolors = [
    "tab:grey",
    "tab:orange",
    ColorCode.cyan,
    ColorCode.green,
    "m",
    "y",
    "k",
    "r",
    "b",
    "tab:purple",
]
custom_cycler = cycler(color=myFRLcolors)

fig, axes = plot_concentration_and_spectra(
    [global_result_670.data["670TR1"], global_result_670.data["670TR2"]],
    cycler=custom_cycler,
    das_cycler=custom_cycler,
)


In [ ]:
global_scheme = Scheme(
    model="models/global_step2_model_PSI_TA_SCy6803WL670_with_DOAS.yml",  # type: ignore
    parameters=load_parameters(
        "models/global_step2_parameters_PSI_TA_SCy6803WL670_with_DOAS.csv"
    ),
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH_670_1,
        "670TR2": DATA_PATH_670_2,
    },  # type: ignore
)
global_scheme.validate()


In [ ]:
global_result670 = optimize(global_scheme, raise_exception=True)


## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
from custom_plotting import plot_residual_and_svd

fig, axes = plot_residual_and_svd([global_result670.data["670TR1"]])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


## Plot result for interpretation


In [ ]:
myFRLcolors = [
    "tab:grey",
    "tab:orange",
    ColorCode.cyan,
    ColorCode.green,
    "m",
    "y",
    "k",
    "r",
    "b",
    "tab:purple",
]

custom_cycler = cycler(color=myFRLcolors)
fig, axes = plot_concentration_and_spectra(
    [global_result670.data["670TR1"], global_result670.data["670TR2"]],
    cycler=custom_cycler,
    das_cycler=custom_cycler,
)


To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 20MB of disk space per save.

In [ ]:
save_result(
    result=global_result670,
    result_path="results/global670/result.yaml",
    allow_overwrite=True,
)


### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result670
# For easier copy-and-paste try:
# print(global_result)


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result670.optimized_parameters


## what to do if the estimated rate constants are not in decreasing order?
then one should sort the estimated rate constants in decreasing order, and repeat the fit with those new starting values

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result670)


## Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA = (
    global_result670.data["670TR1"],
    global_result670.data["670TR2"],
)
wavelengths = select_plot_wavelengths(global_result_TA, equidistant_wavelengths=True)
fig_traces = plot_fitted_traces(global_result_TA, wavelengths, linlog=True, linthresh=1)


The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).

## Overview 670 exc

In [ ]:
fig_670TR1 = plot_overview(
    global_result670.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig_670TR2 = plot_overview(
    global_result670.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=["y", "g", "tab:orange", "r", "k", "c", "b", "m", "tab:purple"]
    ),
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(
    global_result670.data["670TR1"], time_range=(-0.3, 0.3), figsize=(10, 4)
)
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


## Overview of the estimated DOAS and phases of 670 nm excitation data

In [ ]:
fig, axes = plot_doas(
    global_result670.data["670TR1"],
    damped_oscillation=["osc1"],
    time_range=(-0.3, 0.3),
    spectral=700,
    figsize=(15, 4),
    normalize=False,
    # oscillation_type="sin",
    # center_λ=550,
)

axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
# axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[0].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("D", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("E", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)


# Step 2b Global analysis of 700nm excitation TA of WL-PSI of SCy6803

### Data inspection

In [ ]:
# specify data paths
DATA_PATH_700_3 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revc.ascii"
DATA_PATH_700_4 = "data/SCy6803WL/synWTred670_700nm_exc2RPnocycle1nm_revd.ascii"


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH_700_3,
    nr_of_data_svd_vectors=5,
    linlog=False,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
    title="700 nm excitation Time Range 1",
)
# change color map seismic or bwr
# axes[0].set_cmap('seismic')


In [ ]:
fig, axes = plot_data_overview(
    DATA_PATH_700_4,
    nr_of_data_svd_vectors=5,
    linlog=False,
    linthresh=10,
    cmap="seismic",
    vmin=-7,
    vmax=7,
    use_svd_number=True,
    title="700 nm excitation Time Range 2",
)


## Global Analysis

### Model specification

The model and parameters are defined in each step, as the model and/or parameters may be tweaked in each step.

### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    model="models/global_step3_model_PSI_TA_SCy6803WL700.yml",  # type: ignore
    parameters=load_parameters(
        "models/global_step3_parameters_PSI_TA_SCy6803WL700.csv"
    ),
    maximum_number_function_evaluations=15,
    clp_link_tolerance=0.1,
    data={
        # TA data
        # "670TR1": DATA_PATH1,
        # "670TR2": DATA_PATH2,
        "700TR1": DATA_PATH_700_3,
        "700TR2": DATA_PATH_700_4,
    },  # type: ignore
)
global_scheme.validate()


In [ ]:
global_result700 = optimize(global_scheme, raise_exception=True)


To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 50MB of disk space per save.

In [ ]:
save_result(
    result=global_result700,
    result_path="results/global700/result.yaml",
    allow_overwrite=True,
)


### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result700
# For easier copy-and-paste try:
# print(global_result)


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result700.optimized_parameters


## what to do if the estimated rate constants are not in decreasing order?
then one should sort the estimated rate constants in decreasing order, and repeat the fit with those new starting values

### Amplitude matrices

In [ ]:
show_a_matrixes(global_result700)


### Amplitude matrices

In [ ]:
show_a_matrixes(global_result700)


## Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA_700 = (
    global_result700.data["700TR1"],
    global_result700.data["700TR2"],
)
wavelengths = select_plot_wavelengths(
    global_result_TA_700, equidistant_wavelengths=True
)
fig_traces_700 = plot_fitted_traces(
    global_result_TA_700, wavelengths, linlog=True, linthresh=1
)


The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).

## Overview 700 exc

In [ ]:
custom_cycler = cycler(color=["k", "r", "g", "tab:purple"])
fig_700TR1 = plot_overview(
    global_result700.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=1,
    cycler=custom_cycler,
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


In [ ]:
fig_700TR = plot_overview(
    global_result700.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
    linlog=False,
    linthresh=10,
    cycler=custom_cycler,  # defined in cell above
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(
    global_result700.data["700TR1"], time_range=(-0.3, 0.3), figsize=(10, 4)
)
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


## Plot result for interpretation


In [ ]:
# [ "tab:grey","tab:orange",  ColorCode.cyan, ColorCode.green,"m", "y", "k","r", "b", "tab:purple"]
custom_cycler = cycler(color=["k", "r", "g", "tab:purple"])

fig, axes = plot_concentration_and_spectra(
    [global_result700.data["700TR1"], global_result700.data["700TR2"]],
    cycler=custom_cycler,
    das_cycler=custom_cycler,
)


In [ ]:
# result_dataset.data["670TR1"]
# result_dataset.data["700TR1"]
from custom_plotting import plot_final_and_diff_EADS

fig_EADSdiff, _ = plot_final_and_diff_EADS(
    global_result670.data["670TR1"], global_result700.data["700TR1"], scale=0.85
)


## Residual analysis of the 700 nm excitation TR1 data

In [ ]:
fig, axes = plot_residual_and_svd([global_result700.data["700TR1"]])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)



# Step 2c Simultaneous global analysis of TA of WL-PSI of SCy6803

### Inspect data

## Global Analysis

### Used model and parameters

### Create scheme and optimize it

In [ ]:
global_scheme = Scheme(
    model="models/global_step4_model_PSI_TA_SCy6803WL670and700.yml",  # type: ignore
    parameters=load_parameters(
        "models/global_step4_parameters_PSI_TA_SCy6803WL670and700.csv"
    ),
    maximum_number_function_evaluations=5,
    clp_link_tolerance=0.1,
    data={
        # TA data
        "670TR1": DATA_PATH_670_1,
        "670TR2": DATA_PATH_670_2,
        "700TR1": DATA_PATH_700_3,
        "700TR2": DATA_PATH_700_4,
    },  # type: ignore
)
global_scheme.validate()


In [ ]:
global_result_670_700 = optimize(global_scheme, raise_exception=True)


To save the results of the optimization we can use the `save_result` command.

Because it saves *everything* it consumes about 40MB of disk space per save.

In [ ]:
save_result(
    result=global_result_670_700,
    result_path="results/global670and700/result.yaml",
    allow_overwrite=True,
)


### Results and parameters

In [ ]:
# Just call the result to get the optimization result summary.
global_result_670_700
# For easier copy-and-paste try:
# print(global_result)


In [ ]:
# Access the result's `optimized_parameters` to print a markdown table of the optimized parameters:
global_result_670_700.optimized_parameters


### Amplitude matrices

In [ ]:
show_a_matrixes(global_result_670_700)


## Result plots

<sub>Note: The color scheme of the plots in this notebook may not match published figures.</sub>

## Fit quality

In [ ]:
global_result_TA = (
    global_result_670_700.data["670TR1"],
    global_result_670_700.data["670TR2"],
    global_result_670_700.data["700TR1"],
    global_result_670_700.data["700TR2"],
)
wavelengths = select_plot_wavelengths(global_result_TA, equidistant_wavelengths=True)
fig_TA = plot_fitted_traces(global_result_TA, wavelengths, linlog=True, linthresh=1)


The above command `plot_fitted_traces` is used to plot a selection of traces for a set of wavelengths (autogenerated using the `select_plot_wavelengths` function).
To show to make a manual selection of traces, and 'dress up the plot' see the code below, which reproduces Figure 2 of the iScience paper.

In [ ]:
# Reproduction of Figure 2 of the iScience paper
from pyglotaran_extras.plotting.style import ColorCode as cc
from custom_plotting import plot_fitted_traces_iscience

fig, ax_ = plot_fitted_traces_iscience(
    global_result_TA,
    [685, 700, 720, 760],
    linlog=True,
    linthresh=1,  # published figure uses 0.3 for easthetic reasons, but here 1 looks better
    axes_shape=(2, 2),
    figsize=(6, 4),
    title="",
    per_axis_legend=True,
    cycler=cycler(
        color=[
            cc.grey,
            cc.black,
            cc.grey,
            cc.black,
            cc.orange,
            cc.red,
            cc.orange,
            cc.red,
        ]
    ),
)


## Overview 670 exc

In [ ]:
from pyglotaran_extras.plotting.style import ColorCode

fig_670_TR1 = plot_overview(
    global_result_670_700.data["670TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        color=[
            "tab:grey",
            "tab:orange",
            ColorCode.cyan,
            ColorCode.green,
            "m",
            "y",
            "k",
            "r",
            "b",
            "tab:purple",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)


## Residual analysis of the 670 nm excitation TR1 data

In [ ]:
fig, axes = plot_residual_and_svd([global_result_670_700.data["670TR1"]])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


In [ ]:
global_result_670_700.data["670TR2"]


## Residual analysis of all data

In [ ]:
from custom_plotting import plot_svd_of_residual
fig, axes = plot_svd_of_residual(
    [global_result_670_700.data["670TR1"],
    global_result_670_700.data["670TR2"],
    global_result_670_700.data["700TR1"],
    global_result_670_700.data["700TR2"]],
    linlog=True,
    linthresh=1,
    index=0,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


In [ ]:
global_result_670_700.data["700TR2"].residual.plot(x="time")


In [ ]:
global_result_670_700.data["700TR2"].weighted_residual.plot(x="time")


In [ ]:
global_result_670_700.data["700TR1"].weighted_residual.plot(x="time")


In [ ]:
global_result_670_700.data["700TR2"].residual_right_singular_vectors.isel(
    spectral=slice(56, 72), right_singular_value_index=0
).plot()


In [ ]:
global_result_670_700.data["700TR2"].weighted_residual_right_singular_vectors.isel(
    spectral=slice(56, 72), right_singular_value_index=0
).plot()


In [ ]:
fig, axis = plt.subplots(1, 1, figsize=(3, 2))
global_result_670_700.data["700TR1"].data.plot(
    x="time", ax=axis, vmin=-5, vmax=5, cmap="seismic"
)
axis.set_xlabel("Time (ps)")
axis.set_ylabel("Wavelength (nm)")


In [ ]:
fig, axes = plot_svd_of_residual(
    [global_result_670_700.data["670TR1"],
    global_result_670_700.data["670TR2"],
    global_result_670_700.data["700TR1"],
    global_result_670_700.data["700TR2"]],
    linlog=True,
    linthresh=1,
    index=1,
)
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[0].set_title("residual 2nd LSV")
axes[1].set_title("residual 2nd RSV")


In [ ]:
_ = plot_overview(
    global_result_670_700.data["670TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(
        # color=[ "tab:grey","tab:orange",  "c", "g","m", "y", "k","r", "b", "tab:purple"]
        color=[
            "tab:grey",
            "tab:orange",
            ColorCode.cyan,
            ColorCode.green,
            "m",
            "y",
            "k",
            "r",
            "b",
            "tab:purple",
        ]
    ),
    use_svd_number=True,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
)


## Overview 700 exc

In [ ]:
_ = plot_overview(
    global_result_670_700.data["700TR1"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["k", "r", "g", "tab:purple"]),
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


In [ ]:
_ = plot_overview(
    global_result_670_700.data["700TR2"],
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=2,
    linlog=False,
    linthresh=1,
    cycler=cycler(color=["k", "r", "g", "tab:purple"]),
    use_svd_number=True,
    svd_cycler=PlotStyle().cycler,
)


## Residual analysis of the 700 nm excitation TR2 data

In [ ]:
fig, axes = plot_residual_and_svd([global_result_670_700.data["700TR2"]])
axes[0].annotate("A", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)
axes[2].annotate("C", xy=(-0.1, 1), xycoords="axes fraction", fontsize=16)


## Plot result for interpretation


In [ ]:
myFRLcolors = [
    "tab:grey",
    "tab:orange",
    ColorCode.cyan,
    ColorCode.green,
    "m",
    "y",
    "k",
    "r",
    "b",
    "tab:purple",
]
custom_cycler = cycler(color=myFRLcolors)


fig, axes = plot_concentration_and_spectra(
    [global_result_670_700.data["670TR1"], global_result_670_700.data["670TR2"]], cycler=custom_cycler
)


In [ ]:
myFRLcolors = ["k", "r", "g", "tab:purple"]

custom_cycler = cycler(color=myFRLcolors)


fig, axes = plot_concentration_and_spectra(
    [global_result_670_700.data["700TR1"], global_result_670_700.data["700TR2"]],
    cycler=custom_cycler,
    labels=("D", "E", "F"),
)


## Coherent Artifact


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(
    global_result_670_700.data["670TR1"], time_range=(-0.3, 0.3), figsize=(10, 4)
)
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(
    global_result_670_700.data["700TR1"], time_range=(-0.3, 0.3), figsize=(10, 4)
)
axes[0].set_xlabel("Time (ps)")
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()


## Overview of the estimated DOAS and phases of 670 nm excitation data

In [ ]:
from pyglotaran_extras import plot_doas
from pyglotaran_extras.plotting.style import ColorCode

fig, axes = plot_doas(
    global_result_670_700.data["670TR1"],
    damped_oscillation=["osc1"],
    time_range=(-0.3, 0.3),
    spectral=700,
    figsize=(15, 4),
    normalize=False,
    # oscillation_type="sin",
    # center_λ=550,
)

axes[0].set_xlabel("Time (ps)")
axes[0].axhline(0, color="k", linewidth=1)
axes[1].set_xlabel("Wavelength (nm)")
axes[2].set_xlabel("Wavelength (nm)")
axes[1].set_ylabel("")
axes[1].set_title("DOAS")
# axes[0].annotate("A", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[1].annotate("B", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
# axes[2].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[0].annotate("C", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[1].annotate("D", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)
axes[2].annotate("E", xy=(0.01, 0.89), xycoords="axes fraction", fontsize=16)


In [ ]:
from glotaran.io import save_dataset
from glotaran.utils.io import create_clp_guide_dataset

for species in global_result_670_700.data["670TR2"].species:
    clp_guide = create_clp_guide_dataset(global_result_670_700.data["670TR2"], species.item())
    string_in_string = "guide/global670and700_670TR2_clp_{}.ascii".format(
        species.item()
    )
    save_dataset(clp_guide.data, string_in_string, allow_overwrite=True)
